# Langchain and Anthropic

Claude is chat-based model, meaning it is trained on conversation data. However, it is a text based API, meaning it takes in single string. It expects this string to be in a particular format. This means that it is up the user to ensure that is the case. LangChain provides several utilities and helper functions to make sure prompts that you write - whether formatted as a string or as a list of messages - end up formatted correctly.

Because Claude is chat-based but accepts a string as input, it can be treated as either a LangChain `ChatModel` or `LLM`. This means there are two wrappers in LangChain - ChatAnthropic and Anthropic. It is generally recommended to use the ChatAnthropic wrapper, and format your prompts as ChatMessages (we will show examples of this below). This is because it keeps your prompt in a general format that you can easily then also use with other models (should you want to). However, if you want more fine-grained control over the prompt, you can use the Anthropic wrapper - we will show and example of this as well. The `Anthropic` wrapper however is **deprecated**, as all functionality can be achieved in a more generic way using `ChatAnthropic`.

Ref: <https://python.langchain.com/docs/integrations/platforms/anthropic>

## Best Practices

#### No System Messages

Anthropic models are not trained on the concept of a "system message". We have worked with the Anthropic team to handle them somewhat appropriately (a Human message with an admin tag) but this is largely a hack and it is recommended that you do not use system messages.

#### AI Messages Can Continue

A completion from Claude is a continuation of the last text in the string which allows you further control over Claude's output.

For example, putting words in Claude's mouth in a prompt like this:

>`\n\nHuman: Tell me a joke about bears\n\nAssistant: What do you call a bear with no teeth?`

This will return a completion like this `A gummy bear!` instead of a whole new assistant message with a different random bear joke.

In [ ]:
# !pip install -Uq boto3 anthropic-bedrock langchain rich

In [3]:
%load_ext rich

In [15]:
import warnings

import boto3
from anthropic_bedrock import AI_PROMPT, HUMAN_PROMPT
from langchain.chains import SequentialChain, SimpleSequentialChain
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatAnthropic
from langchain.llms.bedrock import Bedrock
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from operator import itemgetter
from langchain.schema import StrOutputParser, HumanMessage
from langchain.schema.runnable import RunnablePassthrough
from rich import print
from IPython.display import Markdown
from utils import get_inference_parameters

warnings.filterwarnings("ignore")

### Initialize Bedrock LLM

Here we utilize `langchain.llms.bedrock.Bedrock` class to initialize our LLM.

In [4]:
# get bedrock runtime client
client = boto3.client("bedrock-runtime", region_name="us-west-2")

bedrock_model_id = "anthropic.claude-v2"  # Bedrock model_id
model_kwargs = get_inference_parameters("anthropic")  # Model kwargs for Anthropic LLMs
# print(model_kwargs)

bedrock_model = Bedrock(
    client=client, model_id=bedrock_model_id, model_kwargs=model_kwargs
)  # Initalize LLM

#### Creating prompts with ChatPromptTemplate

When working with ChatModels, it is preferred that you design your prompts as `ChatPromptTemplates`.

Here is an example below of doing that:

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        ("human", "Hi! How are you?"),
        ("assistant", "I'm doing well. Thanks for asking."),
        ("human", "Tell me a joke about {topic}"),
    ]
)
print(prompt)

You can then invoke the chain like below:

*NOTE:* The below example is using Langchain Expression Language (LCEL)

Refer to <https://python.langchain.com/docs/expression_language> for more details.

In [ ]:
chain = prompt | bedrock_model
chain.invoke({"topic": "cats"})

### Creating Prompts with ChatPromptTemplate

Below we use `HumanMessage` from `langchain.schema` to construct `messages` for the chat model.

We then create a `ChatPromptTemplate` from these messages to invoke this chain.


*NOTE: As there are no input variables to be passed, we send an empty `input` string*

In [ ]:
messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
prompt = ChatPromptTemplate.from_messages(messages=messages)

chain = prompt | bedrock_model
chain.invoke({"input": ""})

### Creating Prompts with ChatPromptTemplate

We can see that under the hood LangChain is not appending any prefix/suffix to SystemMessage's. This is because `Anthropic` has no concept of `SystemMessage`. 

Anthropic requires all prompts to end with assistant messages. This means if the last message is not an assistant message, the suffix `Assistant:` will automatically be inserted.

If you decide instead to use a normal `PromptTemplate` (one that just works on a single string) then we need do the following:

- Prefix our prompt string with `HUMAN_PROMPT` and suffix with `AI_PROMPT`

In [ ]:
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
# Format prompt for Anthropic
anthropic_prompt = PromptTemplate.from_template(
    f"{HUMAN_PROMPT}{prompt.template}{AI_PROMPT}"
)
print(anthropic_prompt)

# Invoke the Chain
chain = anthropic_prompt | bedrock_model
chain.invoke({"topic": "otters"})

## Sequential calls to LLMs with LCEL

In [5]:
from langchain.schema import StrOutputParser

llm = bedrock_model

synopsis_prompt = PromptTemplate.from_template(
    """You are a playwright. Given the <title> of play, it is your job to write a synopsis for that title.

<title>{title}</title>

Playwright: This is a synopsis for the above play:"""
)

review_prompt = PromptTemplate.from_template(
    """You are a play critic from the New York Times. Given the <synopsis> of play, it is your job to write a review for that play.

<synposis>{synopsis}</synposis>

Review from a New York Times play critic of the above play:"""
)

In [7]:
sequential_chain = (
    {"synopsis": synopsis_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)

In [12]:
output = sequential_chain.invoke({"title": "Romance at sunset on the beach"})
Markdown(output)

 Here is a potential New York Times review for the play "Romance at Sunset on the Beach":

Romance Comes Crashing In at the Shore in 'Sunset on the Beach'

The age-old tale of a chance seaside encounter leading to romance gets a modern treatment in the new play "Romance at Sunset on the Beach," now showing at the Seaside Theater. While the plot may feel familiar, strong acting and clever dialogue lift this play above the typical meet-cute.

Leading lady Sarah delivers an endearing performance as the introspective heroine enjoying a solitary walk on the beach. Her calm is delightfully disrupted when a literal run-in with a handsome stranger named James sends her belongings flying. The pair's immediate chemistry is evident from their witty banter, expertly delivered with comedic timing and flirtatious subtext. 

As the sun begins to set, painting the stage in dazzling hues, the action slows to a heartfelt pace. The audience is drawn in as Sarah and James bare their souls, admitting a sense of destiny in their unlikely meeting. We root for the pair as they share a tentative first kiss under the stars.

While the plot borders on cliché, strong acting and sharp writing give this play an undeniable charm. The beauty of the staging during the sunset scene alone is worth the price of admission. Director Jane Smith clearly took great care to maximize the romanticism of the seaside setting.

For audiences seeking an escape into a classic love story, "Romance at Sunset on the Beach" is a fine choice for an evening's entertainment. The tale may be familiar, but strong performances give it new life. Viewers are sure to leave the theater with a smile, reminiscing about their own chance encounters leading to romance.

## Multiple chains

Runnables can easily be used to string together multiple Chains

In [14]:
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = bedrock_model

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

' Barack Obama nació en Honolulu, Hawái y se crió allí. Más tarde se mudó a Chicago, Illinois de adulto. Honolulu está en Hawái, que es un estado de los Estados Unidos.'

In [16]:
prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [19]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

' Orange is a reddish-yellow color. The flag of the Netherlands consists of three horizontal bands of red, white, and blue.'

### Branching and Merging

You may want the output of one component to be processed by 2 or more other components. RunnableMaps let you split or fork the chain so multiple components can process the input in parallel. Later, other components can join or merge the results to synthesize a final response. This type of chain creates a computation graph that looks like the following:

![Branching and Merging](images/langchain_complex_chain_text.png)

In [20]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | model
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | model
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | model
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | model
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [22]:
output = chain.invoke({"input": "scrum"})
Markdown(output)

 Here is a balanced response on the pros and cons of scrum:

Scrum, like any framework or methodology, has its strengths and weaknesses. On the positive side, scrum can provide helpful structure, transparency, and continuous improvement when implemented well. The short sprints promote urgency and accountability, while rituals like retros foster collaboration and adaptation. Overall, scrum aims to empower teams to self-organize and deliver value quickly. 

However, the prescriptive nature of scrum is not ideal for every team or context. Time-boxed sprints can sometimes undermine quality and flexibility. Strict roles and an over-emphasis on metrics like velocity can be constraining. Daily standups often degrade in effectiveness over time. Distributed and newer teams may struggle with the ceremonies.

Ultimately, scrum is just a tool. Teams should feel free to adapt it as needed rather than follow it dogmatically. The agile values of individuals over process, working software over documentation, customer collaboration and responding to change matter more than rituals or artifacts. Scrum works best when implemented in a flexible way, keeping the focus on continuous improvement and adding value, not blindly adhering to a prescribed formula. With pragmatism and common sense, most teams can get significant benefits from scrum while minimizing any weaknesses or limitations.

## Sequential Chain [LEGACY]

1. Simple Sequential Chain
2. Complex Sequential Chain

### Simple Sequential Chain

**CHAIN_1** -> [*OUTPUT*] -> **CHAIN_2**

In [ ]:
model = ChatAnthropic(anthropic_api_key="q2342a5-34534544")

llm = bedrock_model

# prompt template 1
first_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "What is the best name to describe a company that makes {product}? output just the best name you can think of. Only output one name and nothing else.",
        ),
    ]
)

# Chain 1
s_chain_one = LLMChain(llm=llm, prompt=first_prompt)
# print(s_chain_one)

In [ ]:
# prompt template 1
second_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "Write a 20 words description for the following company: {company_name}",
        ),
    ]
)

# Chain 1
s_chain_two = LLMChain(llm=llm, prompt=second_prompt)
# print(s_chain_two)

In [ ]:
overall_simple_chain = SimpleSequentialChain(
    chains=[s_chain_one, s_chain_two], verbose=True
)

# print(overall_simple_chain)

#### Run simple sequential chain

In [ ]:
product = "Queen Size Sheet Set"
overall_simple_chain.run(product)

### Complex Sequential Chain

In [ ]:
# input = "La nourriture était médiocre. J'ai aimé le steak et la sauce qui l'accompagne. A part ça, tout le reste n'était que moi."
input = "Das Essen war mittelmäßig. Mir gefielen das Steak und die dazugehörige Soße. Ansonsten war alles andere nur mittelmäßig."

In [ ]:
first_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Translate the following review to english:" "\n\n{review}"),
    ]
)

chain_one = LLMChain(llm=bedrock_model, prompt=first_prompt, output_key="eng_review")

# print(chain_one)

In [ ]:
second_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "Can you summarize the following review in 1 sentence:" "\n\n{eng_review}",
        ),
    ]
)

# chain 2: input= eng_review and output= summary
chain_two = LLMChain(llm=bedrock_model, prompt=second_prompt, output_key="summary")
# print(chain_two)

In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What language is the following review:\n\n{eng_review}"),
    ]
)

# chain 3: input= Review and output= language
chain_three = LLMChain(llm=bedrock_model, prompt=third_prompt, output_key="language")
# print(chain_three)

In [ ]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "Write a follow up response to the following "
            "summary in the specified language:"
            "\n\nSummary: {summary}\n\nLanguage: {language}",
        ),
    ]
)

# chain 3: input= Review and output= language
chain_four = LLMChain(
    llm=bedrock_model, prompt=fourth_prompt, output_key="followup_message"
)
# print(chain_four)

In [ ]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["eng_review", "summary", "followup_message"],
    verbose=True,
)
# print(overall_chain)

In [ ]:
input

In [ ]:
overall_chain(input)